In [ ]:
from pydrake.all import FindResourceOrThrow, StartMeshcat, MeshcatVisualizer
meshcat = StartMeshcat()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time
from pydrake.math import RigidTransform
import pydot
from IPython.display import SVG, display


from pydrake.all import (
    DiagramBuilder, Simulator, FindResourceOrThrow, MultibodyPlant, PiecewisePolynomial, SceneGraph,
    Parser, MultibodyPositionToGeometryPose, TrajectorySource, Demultiplexer, ConstantVectorSource,
    AddMultibodyPlantSceneGraph, HalfSpace, CoulombFriction
)


def vis(x_traj, tf, time_step=0.01):
    # Create a MultibodyPlant for the arm
    file_name = "planar_walker.urdf"
    builder = DiagramBuilder()
    scene_graph = builder.AddSystem(SceneGraph())
    plant = MultibodyPlant(0.0)
    plant.RegisterAsSourceForSceneGraph(scene_graph)
    Parser(plant=plant).AddModels(file_name)

    plant.WeldFrames(
                plant.world_frame(),
                plant.GetBodyByName("base").body_frame(),
                RigidTransform.Identity()
            )

    plant.Finalize()

    n_q = plant.num_positions()
    n_v = plant.num_velocities()
    n_u = plant.num_actuators()

    x_traj_source = builder.AddSystem(TrajectorySource(x_traj))
    # u_traj_source = builder.AddSystem(TrajectorySource(u_traj))

    demux = builder.AddSystem(Demultiplexer(np.array([n_q, n_v])))
    to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant))
    # zero_inputs = builder.AddSystem(ConstantVectorSource(np.zeros(n_u)))

    # builder.Connect(zero_inputs.get_output_port(), plant.get_actuation_input_port())
    builder.Connect(x_traj_source.get_output_port(), demux.get_input_port())
    builder.Connect(demux.get_output_port(0), to_pose.get_input_port())
    builder.Connect(to_pose.get_output_port(), scene_graph.get_source_pose_port(plant.get_source_id()))
    # builder.Connect(scene_graph.get_query_output_port(), plant.get_geometry_query_input_port())


    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    diagram.set_name("diagram")

    #display(SVG(pydot.graph_from_dot_data(
    #    diagram.GetGraphvizString(max_depth=2))[0].create_svg()))


    # Visualize the motion for `n_playback` times
    # n_playback = 3
    # for i in range(n_playback):
    #   # Set up a simulator to run this diagram.
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(time_step)
    simulator.AdvanceTo(tf);

## Swing Leg

In [ ]:
import importlib
import traj_optim
import fsm_utils

importlib.reload(traj_optim)
importlib.reload(fsm_utils)
from traj_optim import TrajectoryOptimizationSolution, read_csv
from fsm_utils import (
    LEFT_STANCE, RIGHT_STANCE, SPACE_STANCE, DOUBLE_SUPPORT
)

# Walking

# Blue is Right feet, Red is left

solver = TrajectoryOptimizationSolution(n_foot=3)
N = 10
mode_seqs = [RIGHT_STANCE] #, LEFT_STANCE, RIGHT_STANCE]
repeat = 1
initial_states = read_csv("q&v.csv", N * repeat * len(mode_seqs))
tf = 0.5 # just now is tf = 2
destination = 0.3

x_traj, u_traj, prog = solver.solve(N, 
                                    mode_seqs, 
                                    repeat, 
                                    initial_states,
                                     mu=1, 
                                     tf=tf, 
                                     destination=destination, 
                                     iters=2e4,
                                     test=True)

In [ ]:
vis(x_traj, tf)

## Walking

In [ ]:
%tb

import importlib
import traj_optim
import fsm_utils

importlib.reload(traj_optim)
importlib.reload(fsm_utils)
from traj_optim import TrajectoryOptimizationSolution, read_csv
from fsm_utils import (
    LEFT_STANCE, RIGHT_STANCE, SPACE_STANCE, DOUBLE_SUPPORT
)


solver = TrajectoryOptimizationSolution(n_foot=3)
N = 4
mode_seqs = [LEFT_STANCE, RIGHT_STANCE] #, LEFT_STANCE, RIGHT_STANCE]
repeat = 8
n_mode = len(mode_seqs)
#initial_states = read_csv("q&v.csv", N * repeat * len(mode_seqs))
xl = np.array(
        [-1.42628695e-01, 7.91008947e-01, -1.42999388e-01,
          -7.89749727e-01, 1.31650877e+00, -2.43590062e-01, 1.04394874e+00, 
          0, 0, 0, 
          0, 0, 0, 0]
      )
xr = np.array(
          [-1.42628695e-01, 7.91008947e-01, -1.42999388e-01,
         -2.43590062e-01, 1.04394874e+00,  -7.89749727e-01, 1.31650877e+00, 
          0, 0, 0, 
          0, 0, 0, 0]
)

initial_guesses = np.concatenate([xl if (i // N) % n_mode == 0 else xr 
                                  for i in range(n_mode * N * repeat)]).reshape(n_mode * N * repeat, 14)
initial_guesses[:, 0] = np.linspace(xl[0], destination, n_mode * N * repeat)
tf = 1.2 # just now is tf = 2
destination = 2.0

x_traj, u_traj, prog = solver.solve(N, 
                                    mode_seqs, 
                                    repeat, 
                                    initial_guesses,
                                     mu=1, 
                                     tf=tf, 
                                     destination=destination, 
                                     iters=16e4,
                                     test=False)

In [ ]:
vis(x_traj, tf, 0.1)

## Hopping

In [ ]:
%tb

import importlib
import traj_optim
import fsm_utils

importlib.reload(traj_optim)
importlib.reload(fsm_utils)
from traj_optim import TrajectoryOptimizationSolution, read_csv
from fsm_utils import (
    LEFT_STANCE, RIGHT_STANCE, SPACE_STANCE, DOUBLE_SUPPORT
)


solver = TrajectoryOptimizationSolution(n_foot=6)
N = 4
mode_seqs = [RIGHT_STANCE, SPACE_STANCE] #, LEFT_STANCE, RIGHT_STANCE]
repeat = 2
initial_states = read_csv("q&v.csv", N * repeat * len(mode_seqs))
tf = 0.8 # just now is tf = 2
destination = 1.2

x_traj, u_traj, prog = solver.solve(N, 
                                    mode_seqs, 
                                    repeat, 
                                    initial_states,
                                     mu=1, 
                                     tf=tf, 
                                     destination=destination, 
                                     iters=5e4,
                                     test=False)

In [ ]:
vis(x_traj, tf)

## Jumping

In [ ]:
%tb

import importlib
import traj_optim
import fsm_utils

importlib.reload(traj_optim)
importlib.reload(fsm_utils)
from traj_optim import TrajectoryOptimizationSolution, read_csv
from fsm_utils import (
    LEFT_STANCE, RIGHT_STANCE, SPACE_STANCE, DOUBLE_SUPPORT
)


solver = TrajectoryOptimizationSolution(n_foot=6)
N = 4
mode_seqs = [DOUBLE_SUPPORT, SPACE_STANCE] #, LEFT_STANCE, RIGHT_STANCE]
repeat = 2
n_mode = len(mode_seqs)
#initial_states = read_csv("q&v.csv", N * repeat * len(mode_seqs))
initial_state = np.array(
        [-1.42628695e-01, 7.91008947e-01, -1.42999388e-01,
          -7.89749727e-01, 1.31650877e+00, -2.43590062e-01,
          1.04394874e+00, 5.19363218e-03, -1.95243469e+00,
          7.79505381e-03, -1.02836908e+00, 2.33236579e-01,
          2.07434133e+00, -4.95476373e+00]
      )
initial_guesses = np.concatenate([initial_state for _ in range(n_mode * N * repeat)]).reshape(n_mode * N * repeat, 14)
initial_guesses[:, 0] = np.linspace(initial_state[0], destination, n_mode * N * repeat)

tf = 0.8 # just now is tf = 2
destination = 1.6

x_traj_jump, u_traj, prog = solver.solve(N, 
                                    mode_seqs, 
                                    repeat, 
                                    initial_guesses,
                                     mu=1, 
                                     tf=tf, 
                                     destination=destination, 
                                     iters=5e4,
                                     test=False)

In [ ]:
vis(x_traj_jump, tf, time_step=0.1)

## Backflipping

In [ ]:
vis(x_traj, tf)

## Test One

In [ ]:
from pydrake.systems.pyplot_visualizer import PyPlotVisualizer
from pydrake.all import PiecewisePolynomial
from pydrake.geometry import MeshcatAnimation

visualizer = PyPlotVisualizer()
x0 = np.array(
        [-1.42628695e-01, 7.91008947e-01, -1.42999388e-01,
          -7.89749727e-01, 1.31650877e+00, -2.43590062e-01, 1.04394874e+00, 
          0, 0, 0, 
          0, 0, 0, 0]
      )
xflip = np.array(
          [-1.42628695e-01, 7.91008947e-01, -1.42999388e-01,
         -2.43590062e-01, 1.04394874e+00,  -7.89749727e-01, 1.31650877e+00, 
          0, 0, 0, 
          0, 0, 0, 0]
)

file_name = "planar_walker.urdf"
builder = DiagramBuilder()
scene_graph = builder.AddSystem(SceneGraph())
plant = MultibodyPlant(0.0)
plant.RegisterAsSourceForSceneGraph(scene_graph)
Parser(plant=plant).AddModels(file_name)

plant.WeldFrames(
            plant.world_frame(),
            plant.GetBodyByName("base").body_frame(),
            RigidTransform.Identity()
        )

plant.Finalize()
plant_context = plant.CreateDefaultContext()
plant.SetPositionsAndVelocities(plant_context, x0.reshape(-1,1))

n_q = plant.num_positions()
n_v = plant.num_velocities()
n_u = plant.num_actuators()


traj00 = np.vstack(
                    [x0.reshape(-1,14),
                    xflip.reshape(-1,14),
                    x0.reshape(-1,14),
                    xflip.reshape(-1,14),
                    x0.reshape(-1,14),
                    xflip.reshape(-1,14),
                    ]
                    )
traj00[:, 0] = np.linspace(x0[0], 3, 6)

x_traj00 = PiecewisePolynomial.ZeroOrderHold([0, 1, 2, 3, 4, 5], 
                                   traj00.T
                                           )
x_traj_source = builder.AddSystem(TrajectorySource(x_traj00))

demux = builder.AddSystem(Demultiplexer(np.array([n_q, n_v])))
to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(plant))

builder.Connect(x_traj_source.get_output_port(), demux.get_input_port())
builder.Connect(demux.get_output_port(0), to_pose.get_input_port())
builder.Connect(to_pose.get_output_port(), scene_graph.get_source_pose_port(plant.get_source_id()))


MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)


diagram = builder.Build()
diagram.set_name("diagram")

ani = MeshcatAnimation()
simulator = Simulator(diagram)
simulator.Initialize()
simulator.set_target_realtime_rate(1)
simulator.AdvanceTo(5.3);
